# 01 Data Quality Assessment

This notebook evaluates the data quality of the raw_credit_applications.json dataset.

The objectives are to:

- Identify data quality issues  
- Quantify the extent of each issue   
- Prepare remediation steps for the cleaning pipeline  

In [32]:
from pathlib import Path
import sys

# Get project root (one level above notebooks/)
project_root = Path().resolve().parent

# Add project root to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print("Project root:", project_root)
print("src exists:", (project_root / "src").exists())

Project root: /Users/fermisun/Desktop/Nova Vår 2026/Data Governance and Ecosystems/dego-project-teamTXA1
src exists: True


In [33]:
from src.data_loading import load_raw_data

df = load_raw_data()
df.shape

(502, 21)

In [34]:
df
df.to_csv("full_dataset_view.csv", index=False)

In [35]:
import pandas as pd

total_rows = len(df)

duplicate_mask = df.duplicated(subset="_id", keep="first")

duplicate_count = duplicate_mask.sum()
duplicate_percent = duplicate_count / total_rows * 100

duplicate_summary = pd.DataFrame({
    "count": [duplicate_count],
    "percent": [f"{duplicate_percent:.2f} %"]
}, index=["duplicate rows"])

duplicate_summary

,count,percent
duplicate rows,2,0.40 %


In [36]:
df.head()
df.dtypes

_id                                  object
spending_behavior                    object
processing_timestamp                 object
applicant_info.full_name             object
applicant_info.email                 object
applicant_info.ssn                   object
applicant_info.ip_address            object
applicant_info.gender                object
applicant_info.date_of_birth         object
applicant_info.zip_code              object
financials.annual_income             object
financials.credit_history_months      int64
financials.debt_to_income           float64
financials.savings_balance            int64
decision.loan_approved                 bool
decision.rejection_reason            object
loan_purpose                         object
decision.interest_rate              float64
decision.approved_amount            float64
financials.annual_salary            float64
notes                                object
dtype: object

We can see that `financial_income` has the wrong data type and needs to be converted to `float`.

The `financial_salary` column will be merged into the `income` column, after which `financial_salary` will be removed from the dataset.


In [37]:
import pandas as pd

missing_counts = df.isna().sum().sort_values(ascending=False)
missing_percent = (df.isna().mean() * 100).sort_values(ascending=False)

missing_summary = (
    missing_counts.to_frame("missing_count")
    .join(missing_percent.to_frame("missing_percent"))
)

# Add percentage sign
missing_summary["missing_percent"] = (
    missing_summary["missing_percent"]
    .round(2)
    .astype(str) + " %"
)

missing_summary[missing_summary["missing_count"] > 0].head(20)

,missing_count,missing_percent
notes,500,99.6 %
financials.annual_salary,497,99.0 %
loan_purpose,452,90.04 %
processing_timestamp,440,87.65 %
decision.rejection_reason,292,58.17 %
decision.approved_amount,210,41.83 %
decision.interest_rate,210,41.83 %
financials.annual_income,5,1.0 %
applicant_info.ip_address,5,1.0 %
applicant_info.ssn,5,1.0 %


In [38]:
gender_counts = df["applicant_info.gender"].value_counts(dropna=False)

gender_percent = (
    df["applicant_info.gender"]
    .value_counts(normalize=True, dropna=False) * 100
)

gender_summary = pd.DataFrame({
    "count": gender_counts,
    "percent": gender_percent.round(2).astype(str) + " %"
})

gender_summary

,count,percent
applicant_info.gender,,
Male,195,38.84 %
Female,193,38.45 %
F,58,11.55 %
M,53,10.56 %
,2,0.4 %
NaN,1,0.2 %


In [39]:
col = "applicant_info.date_of_birth"

pattern_counts = (
    df[col]
    .dropna()
    .astype(str)
    .str.replace(r"\d", "D", regex=True)
    .value_counts()
)

pattern_percent = pattern_counts / pattern_counts.sum() * 100

date_pattern_summary = pd.DataFrame({
    "count": pattern_counts,
    "percent": pattern_percent.round(2).astype(str) + " %"
})

date_pattern_summary

,count,percent
applicant_info.date_of_birth,,
DDDD-DD-DD,340,67.86 %
DD/DD/DDDD,101,20.16 %
DDDD/DD/DD,56,11.18 %
,4,0.8 %


In [40]:
import pandas as pd

col = "financials.credit_history_months"

history_num = pd.to_numeric(df[col], errors="coerce")

invalid_mask = history_num < 0

invalid_count = invalid_mask.sum()
invalid_percent = invalid_count / len(df) * 100

invalid_summary = pd.DataFrame({
    "count": [invalid_count],
    "percent": [f"{round(invalid_percent, 2)} %"]
}, index=["credit_history_months < 0"])

invalid_summary

,count,percent
credit_history_months < 0,2,0.4 %


In [41]:
import pandas as pd

col = "financials.savings_balance"

savings_num = pd.to_numeric(df[col], errors="coerce")

invalid_mask = savings_num < 0

invalid_count = invalid_mask.sum()
invalid_percent = invalid_count / len(df) * 100

invalid_summary = pd.DataFrame({
    "count": [invalid_count],
    "percent": [f"{round(invalid_percent, 2)} %"]
}, index=["savings_balance < 0"])

invalid_summary

,count,percent
savings_balance < 0,1,0.2 %


Propose and demonstrate remediation steps

In [42]:
#Duplicates
# Copy original dataframe
df_clean = df.copy()

# Count duplicates BEFORE cleaning
duplicates_before = df_clean.duplicated(subset="_id", keep="first").sum()

# Remove duplicates
df_clean = df_clean.drop_duplicates(subset="_id", keep="first")

# Count duplicates AFTER cleaning
duplicates_after = df_clean.duplicated(subset="_id", keep="first").sum()

print("Duplicates before cleaning:", duplicates_before)
print("Duplicates after cleaning:", duplicates_after)


Duplicates before cleaning: 2
Duplicates after cleaning: 0


In [43]:
df_clean["applicant_info.gender"] = (
    df_clean["applicant_info.gender"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({
        "m": "male",
        "f": "female",
        "nan": None,     # convert string "nan" back to missing
        "": None         # convert empty string to missing
    })
)

# Drop rows where gender is missing after cleaning
df_clean = df_clean.dropna(subset=["applicant_info.gender"])

df_clean["applicant_info.gender"].value_counts(dropna=False)

applicant_info.gender
female    251
male      247
Name: count, dtype: int64

In [44]:
col = "applicant_info.date_of_birth"

s = df[col].astype("string").str.strip()

import pandas as pd

# Important: explicitly set dtype to datetime
parsed = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")

# 1) YYYY/MM/DD
mask_year_slash = s.str.match(r"^\d{4}/\d{2}/\d{2}$", na=False)

parsed.loc[mask_year_slash] = pd.to_datetime(
    s.loc[mask_year_slash],
    format="%Y/%m/%d",
    errors="coerce",
)

# 2) XX/XX/XXXX → assume MONTH/DAY/YEAR
mask_us_slash = s.str.match(r"^\d{2}/\d{2}/\d{4}$", na=False)

parsed.loc[mask_us_slash] = pd.to_datetime(
    s.loc[mask_us_slash],
    format="%d/%m/%Y",   # 👈 month/day/year
    errors="coerce",
)

# 3) YYYY-MM-DD
mask_iso_hyphen = s.str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)

parsed.loc[mask_iso_hyphen] = pd.to_datetime(
    s.loc[mask_iso_hyphen],
    format="%Y-%m-%d",
    errors="coerce",
)

# 2b) Re parse remaining slash dates as month/day/year for the tricky ones
remaining_slash = (
    parsed.isna()
    & s.notna()
    & s.str.match(r"^\d{2}/\d{2}/\d{4}$", na=False)
)

parsed.loc[remaining_slash] = pd.to_datetime(
    s.loc[remaining_slash],
    format="%m/%d/%Y",
    errors="coerce",
)

# 4) Standardize format
df_clean[col] = parsed.dt.strftime("%Y-%m-%d")

The `applicant_info.date_of_birth` column contains dates stored as strings in multiple formats.  
Our goal is to convert all values into a consistent `YYYY-MM-DD` format.

What the code does:

1. **Prepare the column**
   - Convert values to string and remove whitespace.
   - Create an empty `datetime64[ns]` Series initialized with `NaT` to store parsed results.

2. **Parse known date formats separately**
   - `YYYY/MM/DD`
   - `DD/MM/YYYY`
   - `YYYY-MM-DD` (ISO format)

   We use regular expressions to identify each format and `pd.to_datetime()` with an explicit `format` argument to ensure precise parsing.

3. **Handle ambiguous slash dates**
   Dates like `05/06/1998` are ambiguous when both day and month are below 12.  
   After the first parsing attempt, any remaining unparsed slash dates are re-parsed using `MM/DD/YYYY` as a fallback.

4. **Standardize the output**
   All successfully parsed dates are converted to the uniform string format `YYYY-MM-DD`.

Result

The column is transformed from inconsistent string representations into a clean, standardized date format suitable for analysis.

In [45]:
df_clean.to_csv("clean_dataset_view.csv", index=False)

In [46]:
df_clean = df_clean.copy()

income_col = "financials.annual_income"
salary_col = "financials.annual_salary"

# Ensure both are numeric
df_clean[income_col] = pd.to_numeric(df_clean[income_col], errors="coerce")
df_clean[salary_col] = pd.to_numeric(df_clean[salary_col], errors="coerce")

# Count how many salary values exist
salary_non_null = df_clean[salary_col].notna().sum()

# Move salary into income ONLY where income is missing
mask_move = df_clean[income_col].isna() & df_clean[salary_col].notna()

moved_count = mask_move.sum()

df_clean.loc[mask_move, income_col] = df_clean.loc[mask_move, salary_col]

# Drop salary column
df_clean = df_clean.drop(columns=[salary_col])

print("Salary values found:", salary_non_null)
print("Values moved to annual_income:", moved_count)

Salary values found: 5
Values moved to annual_income: 5


In [47]:
df_clean.to_csv("clean_dataset_view.csv", index=False)

Sat invalid values to Nan, not sure if good to delete entire row?

In [48]:
import numpy as np

cols = ["financials.savings_balance", "financials.credit_history_months"]

df_clean.loc[df_clean["financials.savings_balance"] < 0, "financials.savings_balance"] = np.nan
df_clean.loc[df_clean["financials.credit_history_months"] < 0, "financials.credit_history_months"] = np.nan


In [49]:
df_clean.to_csv("clean_dataset_view.csv", index=False)

In [50]:
#datetime change?
# Convert processing_timestamp to datetime
df_clean["processing_timestamp"] = pd.to_datetime(
    df_clean["processing_timestamp"],
    errors="coerce"
)

# Convert date_of_birth to datetime
df_clean["applicant_info.date_of_birth"] = pd.to_datetime(
    df_clean["applicant_info.date_of_birth"],
    errors="coerce"
)

print(df_clean["processing_timestamp"].dtype)
print(df_clean["applicant_info.date_of_birth"].dtype)

datetime64[ns, UTC]
datetime64[ns]


In [51]:
df_clean.to_csv("clean_dataset_view.csv", index=False)

still two applications with missing birth date